In [22]:
#################################################
#created the 26/04/2018 13:42 by Alexis Blanchet#
#################################################


import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import scipy.stats
import matplotlib
import plotly.offline as offline
import plotly.graph_objs as go
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)

In [2]:
CHAINE = '192'
DATE = '2017-12-01'
JOINDATE = "".join(list(DATE.split('-')))

c = list(CHAINE)
n = 4-len(c)
c = ['0']*n
a = "".join(c)
CHAINE2 = a+CHAINE

In [3]:
X_PTV = pd.read_csv('PTV/IPTV_'+CHAINE2+'_'+DATE+'_TF1.csv',index_col=False).drop(['Unnamed: 0'],axis=1)
X_RTS = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-processed.csv',index_col=False)
y = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-y.csv',index_col=False)

In [4]:
X_PTV

,DATEHEURE,REDIF,...,HEURE,debut
0,2017-12-01T06:25:00,0,...,06:25:00,385
1,2017-12-01T08:25:00,0,...,08:25:00,505
2,2017-12-01T08:30:00,0,...,08:30:00,510
3,2017-12-01T09:15:00,0,...,09:15:00,555
4,2017-12-01T09:20:00,1,...,09:20:00,560
...,...,...,...,...,...
24,2017-12-01T20:50:00,0,...,20:50:00,1250
25,2017-12-01T21:00:00,0,...,21:00:00,1260
26,2017-12-01T23:30:00,0,...,23:30:00,1410
27,2017-12-02T01:35:00,0,...,01:35:00,95


In [5]:
X_RTS.head()
X_RTS['minutes'] = X_RTS['minutes']+180

In [6]:
X_RTS[-100:]

,t-3,t-2,...,skewness,label
1337,29802021.0,28708364.0,...,4.115200e-26,-0.0
1338,28708364.0,28145216.0,...,9.867282e-27,-0.0
1339,28145216.0,27557702.0,...,5.420800e-27,-0.0
1340,27557702.0,27116627.0,...,1.866477e-27,-0.0
1341,27116627.0,26789023.0,...,2.191086e-28,-0.0
...,...,...,...,...,...
1432,4781302.0,4667193.0,...,5.488208e-31,0.0
1433,4667193.0,4727950.0,...,4.914910e-31,0.0
1434,4727950.0,4799091.0,...,4.236285e-32,-0.0
1435,4799091.0,4767249.0,...,1.659156e-32,-0.0


In [7]:
tdf = pd.DataFrame()
tdf['debut'] = X_PTV['debut']
tdf['fin'] = tdf['debut']+X_PTV['DUREE']
tdf['titre'] = X_PTV['TITRE']
tdf.head()

,debut,fin,titre
0,385,505,TFou
1,505,510,Météo
2,510,555,Téléshopping
3,555,560,Météo
4,560,590,Petits secrets entre voisins


In [8]:
df = pd.DataFrame()
df['minutes'] = (X_RTS['minutes'])
df['isinprogramme'] = 0
df['debut'] = 0
df['fin'] = 0

df.head()
    


,minutes,isinprogramme,debut,fin
0,183,0,0,0
1,184,0,0,0
2,185,0,0,0
3,186,0,0,0
4,187,0,0,0


In [9]:
for index, row in df.iterrows():
    for index2, row2 in tdf.iterrows():        
        if(int(row2['debut']) < int(row['minutes']%1440) < int(row2['fin'])):
            df.set_value(index, 'isinprogramme', 1)
            df.set_value(index, 'fin', row2['fin'])
            df.set_value(index, 'debut', int(row2['debut']))
            df.ix[index,'TITRE']=row2['titre']
            
        if(int(row2['debut']) < int(row['minutes']) < int(row2['fin'])):
            df.set_value(index, 'isinprogramme', 1)
            df.set_value(index, 'fin', row2['fin'])
            df.set_value(index, 'debut', int(row2['debut']))
            df.ix[index,'TITRE']=row2['titre']

        else:
            pass
df[df['isinprogramme']==0]['titre'] = 'en dehors de programmes'

In [10]:
df2 = df

In [11]:
df = df2

In [12]:
tdf.shape

(29, 3)

In [13]:
df.shape

(1437, 5)

In [14]:
df = pd.merge(df, X_RTS, on='minutes')

In [15]:
df[df['isinprogramme']==0]

,minutes,isinprogramme,...,skewness,label
322,505,0,...,4.473009e-26,-0.0
327,510,0,...,1.317887e-26,-0.0
372,555,0,...,9.296079e-28,-0.0
377,560,0,...,1.625372e-28,-0.0
407,590,0,...,2.733815e-28,0.0
...,...,...,...,...,...
1067,1250,0,...,8.703078e-24,-0.0
1077,1260,0,...,1.349010e-20,1.0
1227,1410,0,...,2.874689e-27,-0.0
1352,1535,0,...,1.097693e-28,-0.0


In [16]:
print(df.shape)
df.head()

(1437, 24)


,minutes,isinprogramme,...,skewness,label
0,183,1,...,2.905194e-35,-0.0
1,184,1,...,6.906554e-34,-0.0
2,185,1,...,2.250786e-33,-0.0
3,186,1,...,1.985593e-33,-0.0
4,187,1,...,8.184194e-34,0.0


In [17]:
df = pd.merge(df, X_PTV, on=['debut','TITRE'], how='left')

In [18]:
print(df.shape)
df[:100]

(1437, 59)


,minutes,isinprogramme,...,DATE,HEURE
0,183,1,...,2017-12-02,01:40:00
1,184,1,...,2017-12-02,01:40:00
2,185,1,...,2017-12-02,01:40:00
3,186,1,...,2017-12-02,01:40:00
4,187,1,...,2017-12-02,01:40:00
...,...,...,...,...,...
95,278,1,...,2017-12-02,01:40:00
96,279,1,...,2017-12-02,01:40:00
97,280,1,...,2017-12-02,01:40:00
98,281,1,...,2017-12-02,01:40:00


In [19]:
df = df.drop(['@DATEMODIF','@CLEDIF','DATEHEURE','RATIO'],axis=1)

In [20]:
df[df['isinprogramme']==0]

,minutes,isinprogramme,...,DATE,HEURE
322,505,0,...,NaN,NaN
327,510,0,...,NaN,NaN
372,555,0,...,NaN,NaN
377,560,0,...,NaN,NaN
407,590,0,...,NaN,NaN
...,...,...,...,...,...
1067,1250,0,...,NaN,NaN
1077,1260,0,...,NaN,NaN
1227,1410,0,...,NaN,NaN
1352,1535,0,...,NaN,NaN


In [23]:
df = df.fillna(-1)

In [24]:
df

,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,TEMPSFORT,HD,description programme,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,CSA,@CLEEMI,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,DATE,HEURE
0,183,1,100,390,Programmes de la nuit,3148165.0,3159788.0,3152419.0,3157207,4788.0,-2581.0,9042.0,-7369.0,4254.0,11623.0,3154394.75,2812.25,-1290.5,3014.000000,2127.0,1.865585e-13,0.5,2.905194e-35,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
1,184,1,100,390,Programmes de la nuit,3159788.0,3152419.0,3157207.0,3136355,-20852.0,-16064.0,-23433.0,4788.0,-2581.0,-7369.0,3151442.25,-15087.25,-8032.0,-7811.000000,-1290.5,1.865585e-13,0.5,6.906554e-34,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
2,185,1,100,390,Programmes de la nuit,3152419.0,3157207.0,3136355.0,3123438,-12917.0,-33769.0,-28981.0,-20852.0,-16064.0,4788.0,3142354.75,-18916.75,-16884.5,-9660.333333,-8032.0,1.865585e-13,0.5,2.250786e-33,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
3,186,1,100,390,Programmes de la nuit,3157207.0,3136355.0,3123438.0,3147105,23667.0,10750.0,-10102.0,-12917.0,-33769.0,-20852.0,3141026.25,6078.75,5375.0,-3367.333333,-16884.5,1.865585e-13,0.5,1.985593e-33,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
4,187,1,100,390,Programmes de la nuit,3136355.0,3123438.0,3147105.0,3148270,1165.0,24832.0,11915.0,23667.0,10750.0,-12917.0,3138792.00,9478.00,12416.0,3971.666667,5375.0,1.865585e-13,0.5,8.184194e-34,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,1615,1,100,390,Programmes de la nuit,4781302.0,4667193.0,4727950.0,4799091,71141.0,131898.0,17789.0,60757.0,-53352.0,-114109.0,4743884.00,55207.00,65949.0,5929.666667,-26676.0,1.865585e-13,0.5,5.488208e-31,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
1433,1616,1,100,390,Programmes de la nuit,4667193.0,4727950.0,4799091.0,4767249,-31842.0,39299.0,100056.0,71141.0,131898.0,60757.0,4740370.75,26878.25,19649.5,33352.000000,65949.0,1.865585e-13,0.5,4.914910e-31,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
1434,1617,1,100,390,Programmes de la nuit,4727950.0,4799091.0,4767249.0,4743032,-24217.0,-56059.0,15082.0,-31842.0,39299.0,71141.0,4759330.50,-16298.50,-28029.5,5027.333333,19649.5,1.865585e-13,0.5,4.236285e-32,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,108980346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02,01:40:00
1435,1618,1,100,390,Programmes de la nuit,4799091.0,4767249.0,4743032.0,4788586,45554.0,21337.0,-10505.0,-24217.0,-56059.0,-3

In [26]:
df = df.drop(['HEURE','@CLEEMI'],axis = 1)

In [27]:
df

,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,TEMPSFORT,HD,description programme,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,CSA,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,DATE
0,183,1,100,390,Programmes de la nuit,3148165.0,3159788.0,3152419.0,3157207,4788.0,-2581.0,9042.0,-7369.0,4254.0,11623.0,3154394.75,2812.25,-1290.5,3014.000000,2127.0,1.865585e-13,0.5,2.905194e-35,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
1,184,1,100,390,Programmes de la nuit,3159788.0,3152419.0,3157207.0,3136355,-20852.0,-16064.0,-23433.0,4788.0,-2581.0,-7369.0,3151442.25,-15087.25,-8032.0,-7811.000000,-1290.5,1.865585e-13,0.5,6.906554e-34,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
2,185,1,100,390,Programmes de la nuit,3152419.0,3157207.0,3136355.0,3123438,-12917.0,-33769.0,-28981.0,-20852.0,-16064.0,4788.0,3142354.75,-18916.75,-16884.5,-9660.333333,-8032.0,1.865585e-13,0.5,2.250786e-33,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
3,186,1,100,390,Programmes de la nuit,3157207.0,3136355.0,3123438.0,3147105,23667.0,10750.0,-10102.0,-12917.0,-33769.0,-20852.0,3141026.25,6078.75,5375.0,-3367.333333,-16884.5,1.865585e-13,0.5,1.985593e-33,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
4,187,1,100,390,Programmes de la nuit,3136355.0,3123438.0,3147105.0,3148270,1165.0,24832.0,11915.0,23667.0,10750.0,-12917.0,3138792.00,9478.00,12416.0,3971.666667,5375.0,1.865585e-13,0.5,8.184194e-34,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,1615,1,100,390,Programmes de la nuit,4781302.0,4667193.0,4727950.0,4799091,71141.0,131898.0,17789.0,60757.0,-53352.0,-114109.0,4743884.00,55207.00,65949.0,5929.666667,-26676.0,1.865585e-13,0.5,5.488208e-31,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
1433,1616,1,100,390,Programmes de la nuit,4667193.0,4727950.0,4799091.0,4767249,-31842.0,39299.0,100056.0,71141.0,131898.0,60757.0,4740370.75,26878.25,19649.5,33352.000000,65949.0,1.865585e-13,0.5,4.914910e-31,0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
1434,1617,1,100,390,Programmes de la nuit,4727950.0,4799091.0,4767249.0,4743032,-24217.0,-56059.0,15082.0,-31842.0,39299.0,71141.0,4759330.50,-16298.50,-28029.5,5027.333333,19649.5,1.865585e-13,0.5,4.236285e-32,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1057.0,0.0,290.0,0.0,2017-12-02
1435,1618,1,100,390,Programmes de la nuit,4799091.0,4767249.0,4743032.0,4788586,45554.0,21337.0,-10505.0,-24217.0,-56059.0,-31842.0,4774489.50,14096.50,10668.5,-3501.666667,-28029.5,1.865585e-13,0.5,1.659156e-32,-0.0,1.0,0.0,0.0,1.0,Programmes de la nuit,0.0,0.0,0.0,0.0,Programme indéterminé,16.0,0.0,0.0,0.0,0.0,0

In [54]:
df['temps depuis debut'] = 0
df['temps avant fin'] = 0
df['pourcentage déjà vu'] = 0

df['temps depuis debut'] = df[df['isinprogramme'] == 1].apply(lambda row: min(abs(row['minutes'] - row['debut']),abs(row['minutes']%1440 - row['debut'])),axis = 1)
df['temps avant fin'] = df[df['isinprogramme'] == 1].apply(lambda row: min(abs(row['fin'] - row['minutes']),abs(row['fin'] - row['minutes']%1440)),axis = 1)
df = df.fillna(1)
df['pourcentage déjà vu'] = df['temps depuis debut']/df['DUREE']

In [55]:
df[df['isinprogramme'] == 0]

,minutes,isinprogramme,debut,fin,TITRE,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,TITREEMISSION,STM,VM,TEMPSFORT,HD,description programme,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,CSA,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,DATE,temps depuis debut,temps avant fin,pourcentage déjà vu,temps avant la fin
322,505,0,0,0,-1,52690182.0,51682835.0,51022385.0,50377665,-644720.00,-1305170.0,-2312517.0,-660450.0,-1667797.0,-1007347.00,5.144327e+07,-1065601.75,-652585.00,-7.708390e+05,-833898.50,1.865585e-13,0.500000,4.473009e-26,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
327,510,0,0,0,-1,49142236.0,48561080.0,47984769.0,47424492,-560277.00,-1136588.0,-1717744.0,-576311.0,-1157467.0,-581156.00,4.827814e+07,-853652.25,-568294.00,-5.725813e+05,-578733.50,1.865585e-13,0.500000,1.317887e-26,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
372,555,0,0,0,-1,25577646.0,25471432.0,25142943.0,24736932,-406011.00,-734500.0,-840714.0,-328489.0,-434703.0,-106214.00,2.523224e+07,-495306.25,-367250.00,-2.802380e+05,-217351.50,1.865585e-13,0.500000,9.296079e-28,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
377,560,0,0,0,-1,24039124.0,23802626.0,23609203.0,23474214,-134989.00,-328412.0,-564910.0,-193423.0,-429921.0,-236498.00,2.373129e+07,-257077.75,-164206.00,-1.883033e+05,-214960.50,1.865585e-13,0.500000,1.625372e-28,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
407,590,0,0,0,-1,24750757.0,24998521.0,25185805.0,25403072,217267.00,404551.0,652315.0,187284.0,435048.0,247764.00,2.508454e+07,318533.25,202275.50,2.174383e+05,217524.00,1.865585e-13,0.500000,2.733815e-28,0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,1250,0,0,0,-1,285965021.0,280421225.0,278192007.0,278994594,802587.00,-1426631.0,-6970427.0,-2229218.0,-7773014.0,-3498425.14,2.808932e+08,-1898617.75,-713315.50,-2.323476e+06,-3498425.14,1.865585e-13,0.500000,8.703078e-24,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
1077,1260,0,0,0,-1,292948163.0,297546407.0,306293945.0,341421156,3498425.14,43874749.0,48472993.0,8747538.0,13345782.0,3498425.14,3.095524e+08,10000000.00,3498425.14,3.498425e+06,3498425.14,1.865585e-13,0.500001,1.349010e-20,1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
1227,1410,0,0,0,-1,136971140.0,136823875.0,137247006.0,136126188,-1120818.00,-697687.0,-844952.0,423131.0,275866.0,-147265.00,1.367921e+08,-665864.25,-348843.50,-2.816507e+05,137933.00,1.865585e-13,0.500000,2.874689e-27,-0.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1.0,1.0,-1.0,1.0
1352,1535,0,0,0,-1,26661143.0,26379917.0,26278650.0,26160427,-118223.00,-219490.0,-500716.0,-101267.0,-382493.0,-281226.00,2.637003e+07,-209607.25,-109745.00,-1.669053e+05,-191246.50,1.865585e-13

In [56]:
df.describe()

,minutes,isinprogramme,debut,fin,t-3,t-2,t-1,t,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label,REDIF,INEDIT_CRYPTE,DOLBY,STEREO,STM,VM,TEMPSFORT,HD,CLE-FORMAT,DOLBY51,PT1,DIFFERE,OCCULTATIONMOBILE,CSA,DERDIF,INEDIT_EN_CLAIR,PT2,RESUME,VOST,NOUVEAUTE,DIRECT,PT3,CLE-GENRE,PREMDIF,DUREE,CLAIR,temps depuis debut,temps avant fin,pourcentage déjà vu,temps avant la fin
count,1437.000000,1437.000000,1437.000000,1437.000000,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1.437000e+03,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000,1437.000000
mean,901.000000,0.980515,737.359081,860.414057,1.097211e+08,1.097222e+08,1.097233e+08,1.097244e+08,1.014622e+05,2.245182e+03,3.355040e+03,2.294335e+05,2.243330e+03,1.014604e+05,1.097227e+08,2.968896e+05,8.681327e+04,6.692264e+04,8.681235e+04,1.865585e-13,5.000000e-01,1.050679e-22,0.004175,0.247042,-0.019485,-0.019485,0.961030,0.731385,-0.019485,-0.019485,0.762004,10.681976,-0.019485,0.084203,-0.019485,-0.019485,0.762004,-0.019485,0.225470,0.066806,-0.019485,-0.019485,-0.019485,-0.016701,0.179541,680.656924,-0.019485,123.035491,-0.019485,61.840640,61.253305,0.472567,61.253305
std,414.970481,0.138270,447.276424,408.040300,9.545224e+07,9.545101e+07,9.544976e+07,9.544852e+07,1.462337e+06,5.337926e+06,7.263375e+06,2.059206e+06,5.337926e+06,1.462336e+06,9.540563e+07,2.288942e+06,1.439364e+06,1.429933e+06,1.439364e+06,7.783615e-26,2.728115e-07,8.136095e-22,0.413874,0.474488,0.138270,0.138270,0.276541,0.485381,0.138270,0.138270,0.469550,5.948927,0.138270,0.340828,0.138270,0.138270,0.469550,0.138270,0.462333,0.318408,0.138270,0.138270,0.138270,0.148341,0.431747,382.991772,0.138270,92.528913,0.138270,64.139714,63.798603,0.348512,63.798603
min,183.000000,0.000000,0.000000,0.000000,2.140637e+06,2.140637e+06,2.140637e+06,2.140637e+06,-3.498425e+06,-4.089372e+07,-5.021018e+07,-3.498425e+06,-4.089372e+07,-3.498425e+06,2.145253e+06,-3.498425e+06,-3.498425e+06,-3.498425e+06,-3.498425e+06,1.865585e-13,4.999993e-01,2.587483e-37,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,-1.000000,1.000000
25%,542.000000,1.000000,385.000000,505.000000,2.507339e+07,2.507339e+07,2.507339e+07,2.507339e+07,-4.060110e+05,-7.501210e+05,-1.230000e+06,-4.060110e+05,-7.501210e+05,-4.060110e+05,2.495980e+07,-6.081845e+05,-3.750605e+05,-4.100000e+05,-3.750605e+05,1.865585e-13,4.999999e-01,1.210447e-28,-0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,340.000000,0.000000,55.000000,0.000000,16.000000,16.000000,0.240000,16.000000
50%,901.000000,1.000000,780.000000,810.000000,1.012156e+08,1.012156e+08,1.012156e+08,1.012156e+08,2.366700e+04,3.847300e+04,5.618100e+04,2.366700e+04,3.847300e+04,2.366200e+04,1.019998e+08,2.696100e+04,1.923650e+04,1.872700e+04,1.923650e+04,1.865585e-13,5.000000e-01,9.675424e-27,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,12.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,932.000000,0.000000,105.000000,0.000000,40.0000